---
# Parsing CSV And Excel Files

---

## CSV And Excel Files - Structured Data

---

In [6]:
import pandas as pd
import os

In [2]:
os.makedirs("data/structured_files", exist_ok=True)

In [5]:
# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}
data

{'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
 'Category': ['Electronics',
  'Accessories',
  'Accessories',
  'Electronics',
  'Electronics'],
 'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
 'Stock': [50, 200, 150, 75, 100],
 'Description': ['High-performance laptop with 16GB RAM and 512GB SSD',
  'Wireless optical mouse with ergonomic design',
  'Mechanical keyboard with RGB backlighting',
  '27-inch 4K monitor with HDR support',
  '1080p webcam with noise cancellation']}

In [ ]:
# Panda's DataFrame

df = pd.DataFrame(data)
df

,Product,Category,Price,Stock,Description
0,Laptop,Electronics,999.99,50,High-performance laptop with 16GB RAM and 512G...
1,Mouse,Accessories,29.99,200,Wireless optical mouse with ergonomic design
2,Keyboard,Accessories,79.99,150,Mechanical keyboard with RGB backlighting
3,Monitor,Electronics,299.99,75,27-inch 4K monitor with HDR support
4,Webcam,Electronics,89.99,100,1080p webcam with noise cancellation


In [8]:
# Save dataframe as CSV file

df.to_csv('data/structured_files/products.csv', index=False)

In [10]:
# Save as Excel with multiple sheets

with pd.ExcelWriter('data/structured_files/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Prodcut', index=False)

    # Add another sheet
    summary_date = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_date).to_excel(writer, sheet_name="Summary", index=False)

## CSV Processing `Click Here` 👉 [CSV](https://python.langchain.com/docs/integrations/document_loaders/csv/)
### `Click Here` 👉 [CSVLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.csv_loader.CSVLoader.html)

In [11]:
from langchain_community.document_loaders import CSVLoader

### Method 1: CSVLoader - Each row becomes a document


In [18]:
from langchain_community.document_loaders import CSVLoader

print("1️⃣ CSVLoader - Row-based Documents")

csv_loader = CSVLoader(
    file_path='data/structured_files/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"'
    }
)

csv_docs = csv_loader.load()
print(csv_docs)
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\nFirst document")
print(f"Content: {csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

1️⃣ CSVLoader - Row-based Documents
[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 4}, page_content

### Method 2: Custom CSV processing for better control


In [20]:
df

,Product,Category,Price,Stock,Description
0,Laptop,Electronics,999.99,50,High-performance laptop with 16GB RAM and 512G...
1,Mouse,Accessories,29.99,200,Wireless optical mouse with ergonomic design
2,Keyboard,Accessories,79.99,150,Mechanical keyboard with RGB backlighting
3,Monitor,Electronics,299.99,75,27-inch 4K monitor with HDR support
4,Webcam,Electronics,89.99,100,1080p webcam with noise cancellation


In [23]:
from typing import List
from langchain_core.documents import Document

print("\n2️⃣ Custom CSV Processing")
def process_csv_intelligently(filepath: str) -> List[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []

    # Strategy 1: One document per row with structured content
    for idx, row in df.iterrows():
        # Create structured content
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""

        # Create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)

    return documents



2️⃣ Custom CSV Processing


In [24]:
process_csv_intelligently('data/structured_files/products.csv')

[Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structured_files/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_type': 'product_info'}, page_content='Product Informa

## 📊 CSV Processing Strategies

In [ ]:
print("\n📊 CSV Processing Strategies:")
print("\n1. Row-based (CSVLoader):")
print("  ✅ Simple one-row-one-document")
print("  ✅ Good for record lookups")
print("  ❌ Loses table context")

print("\n2. Intelligent Processing:")
print("  ✅ Preserves relationships")
print("  ✅ Creates summaries")
print("  ✅ Rich metadata")
print("  ✅ Better for Q&A")


📊 CSV Processing Strategies:

1. Row-based (CSVLoader):
  ✅ Simple one-row-one-document
  ✅ Good for record lookups
  ❌ Loses table context

2. Intelligent Processing:
  ✅ Preserves relationships
  ✅ Creates summaries
  ✅ Rich metadata
  ✅ Better for Q&A


## Excel Processing

### Method 1: Using pandas for full control